# サーボで扉を開閉する

AIカメラと連携させる前にサーボで扉を開け閉めするプログラムを作りましょう。LEGOとサーボを使って扉をつくりましょう。以下は一例です。２つのサーボを使って扉を開くLEGOを作ってください。

<img src='https://github.com/TomoniCodeAcademy/LEGO-expoert-course/blob/main/img/pictures/HuskyLens_facerecognition_gate.png?raw=true' width=600>

左の扉のサーボはポートAに接続し、右側の扉のサーボはポートBに接続します。

扉を開け閉めするプログラムは以下となります。

In [ ]:
from spike import Motor
import time

# モーターを初期化します。
motorA = Motor('A')
motorB = Motor('B')

while True:

    # open door
    print('door open')
    motorA.start(speed=-3)
    motorB.start(speed=3)
    time.sleep(3)
    motorA.stop()
    motorB.stop()
    time.sleep(2)

    # close door
    print('close open')
    motorA.run_to_position(0,speed=5)
    motorB.run_to_position(0,speed=5)
    time.sleep(2)

# AIカメラと扉開け閉めプログラムの結合

先ほど作成した顔認識プログラムと扉開け閉めプログラムを連携しましょう。顔認識プログラムで、学習した人と判断したら、上記プログラムの扉を開けるコードを実行します。

In [ ]:
import sys
import time
from hub import port      # ポート接続用モジュールを読み込む
from spike import Motor   # モータ用モジュールを読み込む


# HuskyLens用モジュールを読み込む
sys.path.append('/projects/mylib000')
from huskylens_lib import Algo
from huskylens_lib import HuskyLens

# setup serial port
port.F.mode(port.MODE_FULL_DUPLEX)
time.sleep(1)
port.F.baud(9600)

# initialize HusyLens
husky = HuskyLens(port.F)
husky.send_CMD_REQ_ALGO(Algo.FACE_RECOGNITION)


# モーターを初期化します。
motorA = Motor('A')
motorB = Motor('B')


while True:

    # read target info from HuskyLens
    blocks=husky.read_blocks()
    if len(blocks) > 0:
        print('!! find target(face) !!')
        print(blocks)
        type, id, x, y, w, h = blocks[0]
        print(type, id, x, y, w, h)
        if id == 1:
            print('learned face')
            motorA.start(speed=-3)
            motorB.start(speed=3)
            time.sleep(3)
            motorA.stop()
            motorB.stop()
            time.sleep(2)
        else:
            print('unkown face')
        time.sleep(0.5)
    else:
        print('no target(face)')
        # close door
        motorA.run_to_position(0,speed=5)
        motorB.run_to_position(0,speed=5)

プログラムとしてはこれで完成ですが、もう少し効果的に扉を開けたいので、扉を開ける時の効果音を再生させます。プログラムは以下です

In [ ]:
#
#  機能：顔を認識する。知っている人であれば扉を自動で開く
#        知らない人であれば残念な音楽を鳴らす
#  file name:  gate_with_ai_camera.py
#
#
import sys
import time
from hub import port      # ポート接続用モジュールを読み込む
from spike import Motor   # モータ用モジュールを読み込む

from spike import App     # アプリケーション用モジュールを読み込む
app = App()               #  アプリケーションを初期化(音を鳴らすため)


# HuskyLens用モジュールを読み込む
sys.path.append('/projects/mylib000')
from huskylens_lib import Algo
from huskylens_lib import HuskyLens

# setup serial port
port.F.mode(port.MODE_FULL_DUPLEX)
time.sleep(1)
port.F.baud(9600)

# initialize HusyLens
husky = HuskyLens(port.F)
husky.send_CMD_REQ_ALGO(Algo.FACE_RECOGNITION)


# モーターを初期化します。
motorA = Motor('A')
motorB = Motor('B')


while True:

    # read target info from HuskyLens
    blocks=husky.read_blocks()
    if len(blocks) > 0:
        print('!! find target(face) !!')
        print(blocks)
        type, id, x, y, w, h = blocks[0]
        print(type, id, x, y, w, h)
        if id == 1:
            print('learned face')
            motorA.start(speed=-3)
            motorB.start(speed=3)
            app.play_sound('Door Creek 2')
            motorA.stop()
            motorB.stop()
            time.sleep(1)
        else:
            print('unkown face')
            app.play_sound('Lose')
        time.sleep(0.5)
    else:
        print('no target(face)')
        # close door
        motorA.run_to_position(0,speed=5)
        motorB.run_to_position(0,speed=5)